In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 36528179
paper_name = 'zhao_han_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t', header=None, names=['orf','gene','function','growth_Rate','description'])

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 22 x 5


In [7]:
original_data.head()

,orf,gene,function,growth_Rate,description
0,YHR155W,LAM1,Transport,0.060 ± 0.002,Probable role in retrograde transport of stero...
1,YDR326C,LAM2,Transport,0.052 ± 0.001,Probable role in retrograde transport of stero...
2,YNL257C,LAM3,Transport,0.088 ± 0.001,Probable role in retrograde transport of stero...
3,YHR080C,LAM4,Transport,0.054 ± 0.001,Probable role in retrograde transport of stero...
4,YGR197C,SNG1,Transport,0.069 ± 0.002,Regulates phospholipid translocation and cell ...


In [8]:
original_data['orf'] = original_data['orf'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, gene, function, growth_Rate, description]
Index: []


In [13]:
original_data['data'] = original_data['growth_Rate'].apply(lambda x: float(x.split(' ')[0]))

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(22, 1)

In [19]:
wt_growth_rate = 0.049
original_data['data'] = original_data['data'] - wt_growth_rate

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [22239]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,22239
data_type,value
orf,
YCL010C,0.004
YCL046W,0.007
YDL106C,0.024
YDR315C,0.009
YDR326C,0.003


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22239
,data_type,value
gene_id,orf,
504,YCL010C,0.004
533,YCL046W,0.007
1984,YDL106C,0.024
2442,YDR315C,0.009
2453,YDR326C,0.003


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id       22239           
data_type        value     valuez
gene_id orf                      
504     YCL010C  0.004   2.108597
533     YCL046W  0.007   3.690044
1984    YDL106C  0.024  12.651580
2442    YDR315C  0.009   4.744343
2453    YDR326C  0.003   1.581448

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')